Update 20.12.2017.3 - introduced early stopping for LightGBM & updated LightGBM params

Update 21.12.2017 - Introduced stacking & ensemble

Udate 02.01.2018 - restarted working on EDA

Update 09.01.2018 - working on feature generation

Update 15.01.2018 - working on feature hyperparameter optimization

# Final project: predict future sales

This challenge serves as final project for the "How to win a data science competition" Coursera course.
In this competition you will work with a challenging time-series dataset consisting of daily sales data, kindly provided by one of the largest Russian software firms - 1C Company. 

We are asking you to predict total sales for every product and store in the next month. By solving this competition you will be able to apply and enhance your data science skills.

In [ ]:
import pandas as pd
import numpy as np
import os
import gc
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 

pd.set_option('display.max_rows', 600)
pd.set_option('display.max_columns', 50)

import lightgbm as lgb
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from tqdm import tqdm_notebook

from itertools import product


def downcast_dtypes(df):
    '''
        Changes column types in the dataframe: 
                
                `float64` type to `float32`
                `int64`   type to `int32`
    '''
    
    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype == "int64"]
    
    # Downcast
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int32)
    
    return df

In [ ]:
DATA_FOLDER = '../readonly/final_project_data/'

sales    = pd.read_csv(os.path.join(DATA_FOLDER, 'sales_train.csv.gz'))
items           = pd.read_csv(os.path.join(DATA_FOLDER, 'items.csv'))
item_categories = pd.read_csv(os.path.join(DATA_FOLDER, 'item_categories.csv'))
shops           = pd.read_csv(os.path.join(DATA_FOLDER, 'shops.csv'))
train           = pd.read_csv(os.path.join(DATA_FOLDER, 'sales_train.csv.gz'), compression='gzip')
test           = pd.read_csv(os.path.join(DATA_FOLDER, 'test.csv'))

# EDA

Shape of the loaded dataframes

In [ ]:
print ('sales shape %s' % np.str(sales.shape))
print ('items shape %s' % np.str(items.shape))
print ('item_categories shape %s' % np.str(item_categories.shape))
print ('shops shape %s' % np.str(shops.shape))
print ('train shape %s' % np.str(train.shape))
print ('test shape %s' % np.str(test.shape))

Browse data heads to get an idea of the data

In [ ]:
sales.head()

Note. The negative values in item_cnt_day are returned items

In [ ]:
items.head()

In [ ]:
item_categories.head()

In [ ]:
shops.head()

In [ ]:
train.head()

In [ ]:
test.head()

I join the data to get the consolidated base data frame

1st I add category description to the items df

In [ ]:
items_merge = pd.merge(left = items, right = item_categories , left_on = 'item_category_id', right_on = 'item_category_id')

In [ ]:
print (items_merge.head())

Then I add the category to the items sold

In [ ]:
sales_merge = pd.merge(left = sales,right = items_merge, left_on ='item_id', right_on = 'item_id' )

In [ ]:
print (sales_merge.head())

check the time span of the training data

In [ ]:
dates_df = pd.to_datetime(sales_merge['date'],format='%d.%m.%Y')
print ('Sale from %s to %s' % (str(dates_df.min()),str(dates_df.max())))

#### Note. 2 years & 10 months of sales data. I need to predict the sales in November 2015

### Missing Data 

In [ ]:
# missing values?
sales_merge.isnull().sum()

No missing data, all columns have been populated

I visualize how prices are distributed to understand whether there are some dummy values & outliers.

In [ ]:
sales_merge.boxplot(column = 'item_price')      

In [ ]:
plt.scatter(sales_merge.item_category_id,sales_merge.item_price)
plt.show()

What I did is the following:
* 1st I have checked the prices alone & noticed that most prices are < 100000. 
* Then I have checked the distribution of prices over the item categories & observed that the price > 100000 should be an outlier

Now I quantify the items that are extremely highly priced to confirm I can remove them from the data

In [ ]:
sales_merge[sales_merge.item_price > 100000]

In [ ]:
sales_merge[sales_merge.item_category_id == 75]['item_price'].mean()

#### Note. The high price is probably a typo. I updated the data with the mean

In [ ]:
sales_merge.at[2163826, 'item_price'] = 1859

In [ ]:
sales_merge.iloc[2163826]

## Gaio Data Cleansing

In [ ]:
sales.at[2163826, 'item_price'] = 1859
sales = sales[sales.item_cnt_day<=1000]

Now I am going to visualize the sales over the train period by grouping the items sold by month

In [ ]:
aggrMonth = sales_merge.groupby(['date_block_num'])[['item_cnt_day']].sum()

In [ ]:
plt.plot(aggrMonth.item_cnt_day)
plt.title ("Items Sold/Month")
plt.xlabel("month") 
plt.ylabel("items sold") 
plt.rcParams["figure.figsize"] = (20,10)
plt.show()

NOTES: 

* There seems to be some seasonality in the sales, with an indication of simmetry.
* We can notice a sharp increase followed by a sharp decrease around the spikes. This can be exploited by adding lagged features.

# Features generation

### Add 1st the test data

In [ ]:
sales.head()

In [ ]:
all_data_cols = list(sales)
df_test_append = pd.DataFrame(index=test.index, columns=all_data_cols)
df_test_append = df_test_append.fillna(0)
df_test_append['shop_id'] = test['shop_id']
df_test_append['item_id'] = test['item_id']
df_test_append['date_block_num'] = 34

In [ ]:
df_test_append.head()

In [ ]:
sales = pd.concat((sales, df_test_append)).reset_index(drop=True)
print("all_data size is : {}".format(sales.shape))

### Create a DF with all combination of month, shop & item

This is important because in the months we don't have a data for an item store combination, the machine learning algorithm needs to be specifically told that the sales is zero.

In [ ]:
from itertools import product
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))
#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

In [ ]:
sales_merge = sales.groupby(['date_block_num','shop_id','item_id']).agg({'item_cnt_day': 'sum','item_price': np.mean}).reset_index()
sales_merge = pd.merge(grid,sales_merge,on=['date_block_num','shop_id','item_id'],how='left').fillna(0)
# adding the category id too
sales_merge = pd.merge(sales_merge,items,on=['item_id'],how='left')

In [ ]:
sales_merge.head()

### Create the mean encodings 

In [ ]:
for type_id in ['item_id','shop_id','item_category_id']:
    for column_id,aggregator,aggtype in [('item_price',np.mean,'avg'),('item_cnt_day',np.sum,'sum'),('item_cnt_day',np.mean,'avg')]:

        mean_df = sales_merge.groupby([type_id,'date_block_num']).aggregate(aggregator).reset_index()[[column_id,type_id,'date_block_num']]
        mean_df.columns = [type_id+'_'+aggtype+'_'+column_id,type_id,'date_block_num']

        sales_merge = pd.merge(sales_merge,mean_df,on=['date_block_num',type_id],how='left')

In [ ]:
sales_merge.head()

After having create the grid, I add the lags up to 1 year before to leverage the sales seasonality

In [ ]:
cols_to_rename = list(sales_merge.columns[7:]) + ['item_cnt_day']
print (cols_to_rename)

In [ ]:
from tqdm import tqdm_notebook

shift_range = [1, 2, 3, 4, 5, 12]

sales_merge = downcast_dtypes(sales_merge)

for month_shift in tqdm_notebook(shift_range):
    print("passa %s\n" % str(month_shift))
    train_shift = sales_merge[index_cols + cols_to_rename].copy()
    train_shift['date_block_num'] = train_shift['date_block_num'] + month_shift
    foo = lambda x: '{}_lag_{}'.format(x, month_shift) if x in cols_to_rename else x
    train_shift = train_shift.rename(columns=foo)
    sales_merge = pd.merge(sales_merge, train_shift, on=index_cols, how='left').fillna(0)
    del train_shift
    
gc.collect();

In [ ]:
sales_merge.head()

### Fill NaN with 0

In [ ]:
for feat in sales_merge.columns:
    if 'item_cnt' in feat:
        sales_merge[feat]=sales_merge[feat].fillna(0)
    elif 'item_price' in feat:
        sales_merge[feat]=sales_merge[feat].fillna(sales_merge[feat].median())

In [ ]:
sales_merge.head()

### Drop the columns that I am not going to use for training the model

In [ ]:
to_drop_cols = cols_to_rename + ['item_name','item_price']
print (to_drop_cols)

### Get just recent data

In [ ]:
sales_merge = sales_merge[sales_merge['date_block_num']>12]

### Train/Validation/Test split

I will train on the 1st 32 months & validate on the last month

In [ ]:
print ('1st month: %s , last month: %s' % (str(sales_merge['date_block_num'].min()),str(sales_merge['date_block_num'].max())))

In [ ]:
y_train = sales_merge.loc[sales_merge['date_block_num']<33]['item_cnt_day']
y_val =  sales_merge.loc[sales_merge['date_block_num'] == 33]['item_cnt_day']

X_train = sales_merge[sales_merge['date_block_num']<33].drop(to_drop_cols, axis=1)
X_val =  sales_merge[sales_merge['date_block_num']==33].drop(to_drop_cols, axis=1)
X_test = sales_merge[sales_merge['date_block_num']==34].drop(to_drop_cols, axis=1)

X_final_train = sales_merge[sales_merge['date_block_num']<34].drop(to_drop_cols, axis=1)
y_final_train = sales_merge.loc[sales_merge['date_block_num']<34]['item_cnt_day']

### Clip the training target in the range [0,40]

In [ ]:
y_train=np.clip(y_train,0, 40)
y_val=np.clip(y_val,0, 40)
y_final_train = np.clip(y_final_train,0, 40)

# Modeling & Stacking

### Define the loss function - RMSE

In [ ]:
def rmse(X,y):
    return np.sqrt(mean_squared_error(X, y))

### Select most important features with LightGBM

In [ ]:
import lightgbm as lgb
#GAIO Update 20.12.2017.3 - updated lgb params
lgb_params = {
               'feature_fraction': 0.75,
               'metric': 'rmse',
               'nthread':1, 
               'min_data_in_leaf': 2**7, 
               'bagging_fraction': 0.75, 
               'learning_rate': 0.03, 
               'objective': 'mse', 
               'bagging_seed': 2**7, 
               'num_leaves': 2**7,
               'bagging_freq':1,
               'verbose':0,
                'early_stopping_rounds' :1,
                'eval_metric':'rmse'
              }

#GAIO Update 20.12.2017.3 - introduced early stopping having noticed the model overfitting
# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)

model_lgb = lgb.train(lgb_params,                     
                      lgb_train,
                      num_boost_round=500,
                      valid_sets=lgb_eval,
                      early_stopping_rounds=5)

In [ ]:
lgb.plot_importance(model_lgb,importance_type='gain')

### Tune LightGBM with Hyperopt

LightGBM uses the leaf-wise tree growth algorithm, while many other popular tools use depth-wise tree growth. Compared with depth-wise growth, the leaf-wise algorithm can convenge much faster. However, the leaf-wise growth may be over-fitting if not used with the appropriate parameters.

To get good results using a leaf-wise tree, these are some important parameters:

* num_leaves. This is the main parameter to control the complexity of the tree model. Theoretically, we can set num_leaves = 2^(max_depth) to convert from depth-wise tree. However, this simple conversion is not good in practice. The reason is, when number of leaves are the same, the leaf-wise tree is much deeper than depth-wise tree. As a result, it may be over-fitting. Thus, when trying to tune the num_leaves, we should let it be smaller than 2^(max_depth). For example, when the max_depth=6 the depth-wise tree can get good accuracy, but setting num_leaves to 127 may cause over-fitting, and setting it to 70 or 80 may get better accuracy than depth-wise. Actually, the concept depth can be forgotten in leaf-wise tree, since it doesn't have a correct mapping from leaves to depth.
* min_data_in_leaf. This is a very important parameter to deal with over-fitting in leaf-wise tree. Its value depends on the number of training data and num_leaves. Setting it to a large value can avoid growing too deep a tree, but may cause under-fitting. In practice, setting it to hundreds or thousands is enough for a large dataset.
* max_depth. You also can use max_depth to limit the tree depth explicitly.

In [ ]:
from sklearn.metrics import mean_squared_error
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials

def objective(space):
    print(space)
    
    lgb_params = {
               'feature_fraction': 0.75,
               'metric': 'rmse',
               'nthread':1, 
               'max_depth' : int(space['max_depth']),
               'num_leaves': int(space['num_leaves']),
               'min_data_in_leaf': int(space['min_data_in_leaf']), 
               'bagging_fraction': 0.75, 
               'learning_rate': 0.03, 
               'objective': 'mse', 
               'bagging_seed': int(space['bagging_seed']), 
               'bagging_freq':1,
               'verbose':0,
                'early_stopping_rounds' :1,
                'eval_metric':'rmse'
              }
    opt_lgb = lgb.train(lgb_params,                     
                      lgb_train,
                      num_boost_round=500,
                      valid_sets=lgb_eval,
                      early_stopping_rounds=5)
    
    pred = opt_lgb.predict(X_val)
    mse_scr = mean_squared_error(y_val, pred)
    print ("SCORE:", np.sqrt(mse_scr))
    return {'loss':mse_scr, 'status': STATUS_OK }

space ={    
    'max_depth': hp.quniform("x_max_depth", 4, 16, 1),
    'num_leaves': hp.quniform('num_leaves', 10, 140, 10),
    'bagging_seed': hp.quniform('bagging_seed', 10, 140, 10),
    'min_data_in_leaf': hp.quniform('min_data_in_leaf', 20, 140, 10)
    }


trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=100,            
            trials=trials)

Fit the model with optimal parameters & predict the test results

In [ ]:
from hyperopt import space_eval

# Get the values of the optimal parameters
best_params = space_eval(space, best)

# Fit the model with the optimal hyperparamters
lgb_final_train = lgb.Dataset(X_final_train, y_final_train)




In [ ]:
model_lgb = lgb.train(best_params,                     
                      lgb_final_train,
                      num_boost_round=500)

In [ ]:
train_preds = model_lgb.predict(X_train,num_iteration=model_lgb.best_iteration)
rmse_train = rmse(y_train, train_preds)

val_preds = model_lgb.predict(X_val,num_iteration=model_lgb.best_iteration)
rmse_val = rmse(y_val, val_preds)

print('Train RMSE is %f' % rmse_train)
print('Validation RMSE is %f' % rmse_val)

In [ ]:
pred_lgb = model_lgb.predict(X_test)

In [ ]:
#clip the target values in the range 0-20
out_df = pd.DataFrame({'ID': test.ID, 'item_cnt_month': np.clip(pred_lgb,0,20)})
# you could use any filename. We choose submission here
out_df.to_csv('predict_future_prices_20180115.v0.1.csv', index=False)

In [ ]:
ypred = bst.predict(data, num_iteration=bst.best_iteration)

### 1st Level Models

Recall, that we always use first level models to build two datasets: test meta-features and 2-nd level train-metafetures.

### Test meta-features

I train Linear Regression & LightGBM on training & predict on Training & Validation 

In [ ]:
lr = LinearRegression()
lr.fit(X_train.values, y_train)
train_preds = lr.predict(X_train.values)
rmse_lr_train = rmse(y_train, train_preds)

pred_lr_val = lr.predict(X_val.values)
rmse_lr_val = rmse(y_val, pred_lr_val)

print('Train RMSE is %f' % rmse_lr_train)
print('Validation RMSE is %f' % rmse_lr_val)

In [ ]:
train_preds = model_lgb.predict(X_train)
rmse_train = rmse(y_train, train_preds)

val_preds = model_lgb.predict(X_val)
rmse_val = rmse(y_val, val_preds)

print('Train R-squared is %f' % rmse_train)
print('Validation R-squared is %f' % rmse_val)

Now I predict on the test set & concatenate test predictions to get test meta-features.

In [ ]:
pred_lr = lr.predict(X_test.values)
pred_lgb = model_lgb.predict(X_test)
X_test_level2 = np.c_[pred_lr, pred_lgb] 

### GAIO 13-01-18 - Gebruik lgb voorspel 

In [ ]:
#y_test = model_lgb.predict(X_test)
#clip the target values in the range 0-20
out_df = pd.DataFrame({'ID': test.ID, 'item_cnt_month': np.clip(X_test_level2[:,1],0,20)})
# you could use any filename. We choose submission here
out_df.to_csv('predict_future_prices_20180112.v0.2.csv', index=False)

### Train meta-features

Here, we will use duration T equal to month and M=15.
We get predictions (meta-features) from linear regression and LightGBM for months 27, 28, 29, 30, 31, 32. 

In [ ]:
dates = sales_merge['date_block_num']
dates_train = dates[dates <  34]
dates_train_level2 = dates_train[dates_train.isin([27, 28, 29, 30, 31, 32])]

# That is how we get target for the 2nd level dataset
y_train_level2 = y_train[dates_train.isin([27, 28, 29, 30, 31, 32])]

In [ ]:
# And here we create 2nd level feeature matrix, init it with zeros first
X_train_level2 = np.zeros([y_train_level2.shape[0], 2])

# Now fill `X_train_level2` with metafeatures
for cur_block_num in [27, 28, 29, 30, 31, 32]:
    
    print(cur_block_num)
    
    '''
        1. Split `X_train` into parts
           Remember, that corresponding dates are stored in `dates_train` 
        2. Fit linear regression 
        3. Fit LightGBM and put predictions          
        4. Store predictions from 2. and 3. in the right place of `X_train_level2`. 
           You can use `dates_train_level2` for it
           Make sure the order of the meta-features is the same as in `X_test_level2`
    '''      
    X_train_l1 = X_train.loc[dates_train < cur_block_num]
    y_train_l1 = y_train[dates_train < cur_block_num]
    X_test_l1 = X_train.loc[dates_train == cur_block_num]
        
    lr.fit(X_train_l1.values, y_train_l1)
    pred_lr = lr.predict(X_test_l1.values)
    
    model = lgb.train(lgb_params, lgb.Dataset(X_train_l1, label=y_train_l1), 100)
    pred_lgb = model.predict(X_test_l1)
    
    X_train_level2[dates_train_level2 == cur_block_num] = np.c_[pred_lr, pred_lgb] 

# Ensembling

### Simple convex mix

Let's start with simple linear convex mix:

$$
mix= \alpha\cdot\text{linreg_prediction}+(1-\alpha)\cdot\text{lgb_prediction}
$$

We need to find an optimal $\alpha$. And it is very easy, as it is feasible to do grid search. Next, find the optimal $\alpha$ out of `alphas_to_try` array. Remember, that you need to use train meta-features (not test) when searching for $\alpha$. 

In [ ]:
alphas_to_try = np.linspace(0, 1, 1001)

best_alpha = -1
rmse_train_simple_mix = -1000
for current_alpha in alphas_to_try:
    mix = current_alpha * X_train_level2[:,0] + (1 - current_alpha) * X_train_level2[:,1]
    current_rmse = rmse(y_train_level2, mix)
    if current_rmse > rmse_train_simple_mix:
        best_alpha = current_alpha
        rmse_train_simple_mix = current_rmse
        
print('Best alpha: %f; Corresponding RMSE score on train: %f' % (best_alpha, rmse_train_simple_mix))

In [ ]:
test_preds = best_alpha * X_test_level2[:,0] + (1 - best_alpha) * X_test_level2[:,1]

In [ ]:
#y_test = model_lgb.predict(X_test)
#clip the target values in the range 0-20
out_df = pd.DataFrame({'ID': test.ID, 'item_cnt_month': np.clip(test_preds,0,20)})
# you could use any filename. We choose submission here
out_df.to_csv('predict_future_prices_20180112.v0.1.csv', index=False)